## Task

https://en.wikipedia.org/wiki/Nonogram

Given riddle, return possible coloring.

### Solution idea
This solution idea is to use bactracking and inferencing.
First we try to inference about as many fields as possible. When we can't find out anything else we select field without assigned value and assign value to it (first black. then empty if failure). Then we try to inference as much as possible, then we assign value and so on until we find solution or find out that our assignments were wrong and we have to go back.
#### Inferencing
We generate all possible colorings for rows and columns given requirements for them. We try to find fields which are always white or black and use this information further.

### Input
```
20 20
1 1 4
1 6
1 1 1 1 2 3
1 1 2 3
3 1 2 3
4 5 2 2
7 3 2
3 5 1 2
2 2 4 1
2 2 3 4
2 5 2
2 1 5 1
2 2 3 1
6 2 2
1 7
2 2 2
1 4
3 1 1
1 1
1 1
6 1
8 3
3 2 1
1 1 2 2 1
1 2 2 1 1
1 1 1 1
2 3
4 1 2 2
5 2 1
8 1 1
7 2
3 5 2
2 5
2 1 4
2 2 2 2
2 2 1 1 1
3 1 1 1 1
5 4 2 1
7 4 1 1
4
```
[number of rows] [number of columns]

[blocks in row_1]

[blocks in row_2]

$\vdots$

[blocks in column_1]

[blocks in column_2]

$\vdots$

### Output
Solved riddle

`1` for black field

`0` for empty field

## Imports

In [1]:
import numpy as np

## Code

In [2]:
# input

def read_data(file_name='zad_input.txt'):
    data = open('zad_input.txt', 'r')
    data = data.readlines()
    line0 = data[0].split()
    height = int(line0[0])
    width = int(line0[1])
    rows = []
    cols = []
    for i in range(1,height+1):
        specification = data[i].strip().split()
        length = len(specification)
        rows.append(np.zeros(length,dtype=np.int16))
        j=0
        for elem in specification:
            rows[i-1][j]=int(elem)
            j += 1
    for i in range(height+1, 1+height+width):
        specification = data[i].strip().split()
        length = len(specification)
        cols.append(np.zeros(length,dtype=np.int16))
        j=0
        for elem in specification:
            cols[i-height-1][j]=int(elem)
            j+=1
    return height,width,rows,cols

In [3]:
def prog(file_name='zad_input.txt'):
    # input
    height,width,rows,cols = read_data(file_name=file_name)
    
    csp = {'height' : height, 'width': width, 'rows': rows, 'cols': cols}
    assignment = create_empty_assignment(height,width,rows,cols)
    assignment = inference_unpack(assignment,csp)
    return backtrack(assignment,csp)
    
def backtrack(assignment,csp):
    if solved(assignment['board']):
        return assignment
    for i in range(csp['height']):
        for j in range(csp['width']):
            if assignment['board'][i,j]==-1:
                for value in [0,1]:
                    new_assignment =  add_to_assignment(assignment,(i,j),value)
                    new_assignment['board'][i,j]=value
#                     print('stara plansza',assignment['board'],i,j,new_assignment['board'])
                    inferenced_assignment = inference_unpack(new_assignment,csp)
                    if inferenced_assignment is not None:
                        returned_assignment = backtrack(inferenced_assignment,csp)
                        if returned_assignment is not None:
                            return returned_assignment
                return None
    return None

In [4]:
def add_to_assignment(assignment,place,value):
    temp_1 = create_dict(assignment['board'].shape[0])
    temp_1[place[0]].add((place[1],value))
    temp_2 = create_dict(assignment['board'].shape[1])
    temp_2[place[1]].add((place[0],value))
    return {
        'board': assignment['board'].copy(),
        'colorings_rows': assignment['colorings_rows'].copy(),
        'colorings_cols': assignment['colorings_cols'].copy(),
        'dict_rows': temp_1,
        'dict_cols': temp_2
    }

def create_empty_assignment(height,width,rows,cols):
    return {
        'board': np.full([height,width],-1,dtype=np.int8),
        'colorings_rows': create_colorings_for_rows_cols(width,rows),
        'colorings_cols': create_colorings_for_rows_cols(height,cols),
        'dict_rows': create_dict(height),
        'dict_cols': create_dict(width)
    }

def return_assignment(board,colorings_rows,colorings_cols,height,width):
    return {
        'board': board,
        'colorings_rows': colorings_rows,
        'colorings_cols': colorings_cols,
        'dict_rows': create_dict(height),
        'dict_cols': create_dict(width)
    }

In [5]:
def inference_unpack(a,csp):
    ans = inference_solve(csp['height'],csp['width'],csp['rows'],csp['cols'],
                    a['board'],a['colorings_rows'],a['colorings_cols'],
                     a['dict_rows'],a['dict_cols'])
    if ans is None:
        return None
    board,colorings_rows,colorings_cols = ans
    return return_assignment(board,colorings_rows,colorings_cols,csp['height'],csp['width'])

def inference_solve(height,width,rows,cols,board,colorings_rows,colorings_cols,dict_rows,dict_cols):
    flag_rows = True
    for i in range(height):
        colorings_rows[i] = cut(colorings_rows[i],dict_rows[i])
        if colorings_rows[i]==[]:
            return None
        sure = inference(colorings_rows[i])
        for j,elem in sure:
            if board[i,j]==-1:
                board[i,j] = elem
                dict_cols[j].add((i,elem))
    
    for j in range(width):
        colorings_cols[j] = cut(colorings_cols[j],dict_cols[j])
        if colorings_cols[j]==[]:
            return None
        sure = inference(colorings_cols[j])
        for i,elem in sure:
            if board[i,j]==-1:
                board[i,j] = elem
                dict_rows[i].add((j,elem))
    dict_cols = create_dict(width)
    
    changed = True
    while changed:
        changed = False
        if flag_rows:
            for i in range(height):
                if not is_empty_set(dict_rows[i]):
                    colorings_rows[i] = cut(colorings_rows[i],dict_rows[i])
                    if colorings_rows[i]==[]:
                        return None
                    sure = inference(colorings_rows[i])
                    for j,elem in sure:
                        if board[i,j]==-1:
                            changed = True
                            board[i,j] = elem
                            dict_cols[j].add((i,elem))
            dict_rows = create_dict(height)
        else:
            for j in range(width):
                if not is_empty_set(dict_cols[j]):
                    colorings_cols[j] = cut(colorings_cols[j],dict_cols[j])
                    if colorings_cols[j]==[]:
                        return None
                    sure = inference(colorings_cols[j])
                    for i,elem in sure:
                        if board[i,j]==-1:
                            changed = True
                            board[i,j] = elem
                            dict_rows[i].add((j,elem))
            dict_cols = create_dict(width)
        flag_rows = not flag_rows
    return board,colorings_rows,colorings_cols

def create_dict(size):
    d = []
    for i in range(size):
        d.append(set())
    return d

def solved(board):
    return np.all(board!=-1)


In [6]:
def create_colorings_for_rows_cols(size,specs_list):
    colorings_r_c = []
    for specs in specs_list:
        colorings_r_c.append(generate_coloring(size,specs))
    return colorings_r_c

def generate_coloring(size,specs):
    
    def rec_gen_col(i,line,specs,sum_,colorings):
        for j in range(i,np.size(line)-sum_+1):
            line_next = line.copy()
            line_next[j:j+specs[0]].fill(1)
            if np.size(specs)==1:
                colorings.append(line_next)
            else:
                rec_gen_col(j+specs[0]+1,line_next,specs[1:],sum_-specs[0]-1,colorings)
    
    colorings = []
    line = np.zeros(size,dtype=np.int8)
    
    if specs[0]==0:
        colorings.append(line)
    else:
        sum_ = np.size(specs)-1+np.sum(specs)
        rec_gen_col(0,line,specs,sum_,colorings)
    
    return colorings

In [7]:
def inference(colorings):
    size = np.size(colorings[0])
    sure = set()
    for i in range(size):
        elem = colorings[0][i]
        flag = True
        for coloring in colorings[1:]:
            if coloring[i]!=elem:
                flag = False
                break
        if flag==True:
            sure.add((i,elem))
    return sure

def cut(colorings,sure):
    for i,elem in sure:
        new_colorings = []
        for coloring in colorings:
            if coloring[i]==elem:
                new_colorings.append(coloring)
        colorings = new_colorings
    return colorings

In [8]:
def is_empty_set(s):
    return s == set()

In [9]:
prog()['board']

array([[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0,